Import needed libraries

In [1]:
import pandas as pd

Read data

In [2]:
labeled_df = pd.read_feather('hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/labeled_data.feather')

In [3]:
labeled_df.head(n=3)

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0


In [4]:
unlabeled_df = pd.read_feather('hf://datasets/pfacouetey/DSTI_Deep_Learning_Project_2025/unlabeled_data.feather')

In [5]:
unlabeled_df.head(n=3)

,text
0,Any idea what this is? So I came here for awns...
1,"Advice, please. I've been getting so easily en..."
2,Can aggressive behavior in humans be modified ...
